# Utah Teapot Rendering

The code following code has been written by me for a previous project. It contains functions of drawing the Bezier surface for the Utah teapot as given by the TeapotBezier.txt and poly.txt files. The project also required the rotation and zooming of the surface so constructed and is an example of how Bezier surfaces are used to render object shapes and can also be used to manipulate them.

In [2]:
from math import *
import numpy as np
from operator import add
import matplotlib.pyplot as plt
import time
import re
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.backends.backend_agg as agg
import pygame, time
import sys
from numpy import linalg as linalg

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
def plotIt(Points):
	fig, ax = plt.subplots(figsize=(16, 12))

	fig.patch.set_alpha(0.0)
	ax.set_xlabel('x')
	ax.set_ylabel('y')
	ax.set_xlim([-8,8])
	ax.set_ylim([-6,6])
	ax.grid(False)
	ax.axis('off')
	ax.autoscale(False)
	canvas = agg.FigureCanvasAgg(fig)

	ax.plot(Points[:,0],Points[:,1])
	canvas.draw()
	renderer = canvas.get_renderer()

	rawData = renderer.tostring_rgb()
	size = canvas.get_width_height()

	return pygame.image.fromstring(rawData, size, "RGB")

In [4]:
#This Function returns points on a bezier curve
def real(p1,p2,p3,p4):
	v=[]
	mat=[[-1,3,-3,1],[3,-6,3,0],[-3,3,0,0],[1,0,0,0]]
	mat=np.array(mat)
	
	for t in np.arange(0,1.1,0.2):
		
		p=np.array([p1,p2,p3,p4])
		temp=[t**3,t**2,t,1]
		temp=np.array(temp)
		k = np.dot(temp,mat)
		value=np.dot(k,p)
		myFor = [ '%.2f' % elem for elem in value ]
		value=list(map(float,myFor))
		
		v.append(value)

	v = np.asarray(v)
	return v

In [5]:
def surface(data):

	#Reading associations
	f=open("poly.txt","r")
	fr=f.read()
	frl=fr.strip().split("\n")
	data_vertices=[]
	co = []
	for i in frl:
		t=list(map(int,i.strip().split(",")))
		t=[k-1 for k in t]

		data_vertices.append(t)
	for patch in data_vertices:
		u=[]
		v=[]
		pa_points=[]
		# divides a patch into list of p0,p1,p2,p3]p4,p5...
		for i in range(0,len(patch),4):
				u.append([data[patch[i]],data[patch[i+1]],data[patch[i+2]],data[patch[i+3]]])
		# divides a patch into list of p0,p4,p8,p12]p1,p5...
		for i in range(4):
				v.append([data[patch[i]],data[patch[i+4]],data[patch[i+8]],data[patch[i+12]]])
		# drawing bz curves for v for every value in u direction
		x_d=[]
		com_u_d=[]
		for i in u:
			va=real(i[0],i[1],i[2],i[3])
			x_d.append(va)
		y=0
		for i in range(len(x_d[0])):
				# for y in range(4):
				va=real(x_d[y][i],x_d[y+1][i],x_d[y+2][i],x_d[y+3][i])
				co.append(va)
		# drawing bz curves for u for every value in v direction
		v_d=[]
		com_v_d=[]
		for i in v:
			va=real(i[0],i[1],i[2],i[3])
			v_d.append(va)
		y=0
		for i in range(len(v_d[0])):
				# for y in range(4):
				va=real(v_d[y][i],v_d[y+1][i],v_d[y+2][i],v_d[y+3][i])
				co.append(va)


	# com_u_d are curves in u direction and com_v_d are curves in v direction
	co = np.asarray(co)
	

	return co


In [6]:
def Teapot():
	listPoints = []
	
	with open('TeapotBezier.txt') as f:
		i = 1
		for line in f:
			if i==1 :
				patchesNum = int(line)
				i = i+1
			elif i <= 33:
				line = line[:-1]
				i = i+1
				splits = re.split(r",",line)
				abc = np.asarray(list(map(int, splits))).reshape((4,4))
				listPoints.append(abc)
			elif i == 34:
				numPoints = int(line)
				i = i+1
				Points = np.zeros((numPoints,3))
			else:
				splits = re.split(r",",line)
				Points[i-35] = np.asarray(list(map(float, splits)))
				i = i+1
	return Points


In [7]:
def showText():
	text = pygame.font.SysFont("monospace", 15)
	ScreenText = text.render("Press WASD to ROTATE. Press ZX to ZOOM", 4, (1,1,1))
	screen.fill((255,255,255))
	screen.blit(ScreenText, (0, 0))

In [8]:
def zoomIn(pPoints):
	Points = np.ones((pPoints.shape[0],pPoints.shape[1]+1))
	Points[:,:-1] = pPoints
	Zoom = np.array([[1.25,0,0,0],[0,1.25,0,0],[0,0,1.25,0],[0,0,0,1]])
	Points = np.dot(Points,Zoom)
	Points = Points[:,:-1]
	return Points

In [9]:
def zoomOut(pPoints):
	Points = np.ones((pPoints.shape[0],pPoints.shape[1]+1))
	Points[:,:-1] = pPoints
	Zoom = np.array([[0.8,0,0,0],[0,0.8,0,0],[0,0,0.8,0],[0,0,0,1]])
	Points = np.dot(Points,Zoom)
	Points = Points[:,:-1]
	return Points

In [10]:
def cameraPerspective(Points):
	k = 2
	r = 1.0/k
	Tp = np.zeros((4,4))
	Tp[0,0] = 1
	Tp[1,1] = 1
	Tp[2,3] = r
	Tp[3,3] = 1

	pnts = np.ones((Points.shape[0],Points.shape[1]+1))
	pnts[:,:-1] = Points

	pnts = np.dot(pnts,Tp)
	Points = pnts[:,:-1]

	return Points

In [11]:
def cameraTransform(Points,B,D):
	k = 2
	E = np.array([[0,0,-k]]) #To be shifted here. Standard position of the camera
	
	D = D/linalg.norm(D)

	T1 = np.zeros((4,4))
	T1[0,0] = 1
	T1[1,1] = 1
	T1[2,2] = 1
	T1[3,3] = 1
	T1[3,:-1] = -B

	I = np.zeros((3,3))
	I[0,0] = 1
	I[1,1] = 1
	I[2,2] = 1

	u = np.array([np.cross(D,np.array([0,0,1]))])
	u = u/linalg.norm(u)

	cost = np.dot(D,np.array([0,0,1]))
	sint = np.sqrt(1-cost*cost)

	uut = u.T*u
	U = np.array([[0,-u[0,2],u[0,1]],[u[0,2],0,-u[0,0]],[-u[0,1],u[0,0],0]])

	M = uut + cost*(I-uut) + sint*U
	T2 = np.zeros((4,4))
	T2[:-1,:-1] = M.T
	T2[3,3] = 1

	T3 = np.zeros((4,4))
	T3[0,0] = 1
	T3[1,1] = 1
	T3[2,2] = 1
	T3[3,3] = 1
	T3[3,2] = -k

	r = 1.0/k
	Tp = np.zeros((4,4))
	Tp[0,0] = 1
	Tp[1,1] = 1
	Tp[2,3] = r
	Tp[3,3] = 1

	P = np.array([[0,1,10]])
	T = np.dot(T1,T2)
	T = np.dot(T,T3)

	pnts = np.ones((Points.shape[0],Points.shape[1]+1))
	pnts[:,:-1] = Points

	pnts = np.dot(pnts,T)
	Points = pnts[:,:-1]

	return Points



In [12]:
def rotateX(Points,z):
	a = np.cos(z*np.pi/10)
	b = np.sin(z*np.pi/10)

	r = np.array([[1,0,0],[0,a,-b],[0,b,a]])
	Points = np.dot(Points,r)
	return Points


In [13]:
def rotateY(Points,z):
	a = np.cos(z*np.pi/10)
	b = np.sin(z*np.pi/10)

	r = np.array([[a,0,-b],[0,1,0],[b,0,a]])
	Points = np.dot(Points,r)
	return Points


In [14]:

pygame.init()
clock = pygame.time.Clock()
window = pygame.display.set_mode((720,720))
screen = pygame.display.get_surface()
pygame.display.set_caption('Utah Teapot')

text = pygame.font.SysFont("monospace", 15)

#Initialize Teapot Points:
camPos = np.array([0,0,-2])
camDir = np.array([1,1,1])
originalPoints = Teapot()
Points = originalPoints
Points = cameraTransform(Points,camPos,camDir)
curves = surface(Points)
projPoints = cameraPerspective(curves.reshape((curves.shape[0]*curves.shape[1],curves.shape[2])))
showText()
screen.blit(plotIt(projPoints), (20,20))
pygame.display.update()
clock.tick(30)

oriPoints = Points


while True:
    screen.fill((255,255,255))
    for event in pygame.event.get():
        if (event.type == pygame.KEYDOWN):
            if(event.key == pygame.K_ESCAPE):
                sys.exit()
            elif(event.key == ord("r")):
                Points = oriPoints
                curves = surface(Points)
                projPoints = cameraPerspective(curves.reshape((curves.shape[0]*curves.shape[1],curves.shape[2])))
                showText()
                screen.blit(plotIt(projPoints), (20,20))
                pygame.display.update()
                clock.tick(30)
            elif(event.key == ord("w")):
                Points = rotateX(Points,1)
                curves = surface(Points)
                projPoints = cameraPerspective(curves.reshape((curves.shape[0]*curves.shape[1],curves.shape[2])))
                screen.blit(plotIt(projPoints), (20,20))
                pygame.display.update()
                clock.tick(30)
            elif(event.key == ord("s")):
                Points = rotateX(Points,-1)
                curves = surface(Points)
                projPoints = cameraPerspective(curves.reshape((curves.shape[0]*curves.shape[1],curves.shape[2])))
                showText()
                screen.blit(plotIt(projPoints), (20,20))
                pygame.display.update()
                clock.tick(30)
            elif(event.key == ord("a")):
                Points = rotateY(Points,1)
                curves = surface(Points)
                projPoints = cameraPerspective(curves.reshape((curves.shape[0]*curves.shape[1],curves.shape[2])))
                showText()
                screen.blit(plotIt(projPoints), (20,20))
                pygame.display.update()
                clock.tick(30)
            elif(event.key == ord("d")):
                Points = rotateY(Points,-1)
                curves = surface(Points)
                projPoints = cameraPerspective(curves.reshape((curves.shape[0]*curves.shape[1],curves.shape[2])))
                showText()
                screen.blit(plotIt(projPoints), (20,20))
                pygame.display.update()
                clock.tick(30)
            elif(event.key == ord("z")):
                Points = zoomIn(Points)
                curves = surface(Points)
                projPoints = cameraPerspective(curves.reshape((curves.shape[0]*curves.shape[1],curves.shape[2])))
                showText()
                screen.blit(plotIt(projPoints), (20,20))
                pygame.display.update()
                clock.tick(30)
            elif(event.key == ord("x")):
                Points = zoomOut(Points)
                curves = surface(Points)
                projPoints = cameraPerspective(curves.reshape((curves.shape[0]*curves.shape[1],curves.shape[2])))
                showText()
                screen.blit(plotIt(projPoints), (20,20))
                pygame.display.update()
                clock.tick(30)
            elif(event.key == ord("q")):
                pygame.quit()
                sys.exit()
            
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

C:\Users\Mili\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


SystemExit: 

C:\Users\Mili\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
